<a href="https://colab.research.google.com/github/sid-chaubs/data-mining-assignment-1/blob/main/DMT_Assignment_2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/datadrive')

Drive already mounted at /content/datadrive; to attempt to forcibly remount, call drive.mount("/content/datadrive", force_remount=True).


In [2]:
!mkdir /content/training_data
!mkdir /content/test_data
!unzip -o /content/datadrive/MyDrive/training_set_VU_DM.csv.zip -d /content/training_data/
!unzip -o /content/datadrive/MyDrive/test_set_VU_DM.csv.zip -d /content/test_data/

mkdir: cannot create directory ‘/content/training_data’: File exists
mkdir: cannot create directory ‘/content/test_data’: File exists
Archive:  /content/datadrive/MyDrive/training_set_VU_DM.csv.zip
  inflating: /content/training_data/training_set_VU_DM.csv  
Archive:  /content/datadrive/MyDrive/test_set_VU_DM.csv.zip
  inflating: /content/test_data/test_set_VU_DM.csv  


In [63]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
#from yellowbrick import target, features
import joblib
import csv
SEED = 1337 * 420 * 69
SAMPLE_SIZE = 5000

np.random.seed(SEED)

In [6]:
test_data = pd.read_csv('/content/test_data/test_set_VU_DM.csv')

#1 Load trained forests and produce results

In [7]:
#forest_booking = joblib.load('/content/datadrive/MyDrive/forest_booking.joblib')

In [8]:
forest_click = joblib.load('/content/datadrive/MyDrive/forest_click.joblib')

In [9]:
nan_filter = np.sum(test_data.isna()) > 0.75 * test_data.shape[0]
test_data.drop(columns=test_data.columns[nan_filter], inplace=True)
test_data.drop(columns=['date_time'], inplace=True)

In [10]:
rs_params_test = sp.stats.genlogistic.fit(test_data.prop_review_score.dropna()[test_data.prop_review_score > 0])
ls2_params_test = sp.stats.expon.fit(test_data.prop_location_score2.dropna())
dd_params_test = sp.stats.expon.fit(test_data.orig_destination_distance.dropna())

rs_na_count_test = test_data.prop_review_score.isna().sum()
ls2_na_count_test = test_data.prop_location_score2.isna().sum()
dd_na_count_test = test_data.orig_destination_distance.isna().sum()

#generate values from fitted distribution
rs_fill_test = sp.stats.genlogistic.rvs(*rs_params_test, rs_na_count_test)
ls2_fill_test = sp.stats.expon.rvs(*ls2_params_test, ls2_na_count_test)
dd_fill_test = sp.stats.expon.rvs(*dd_params_test, dd_na_count_test)

rs_fill_series_test = pd.Series(rs_fill_test, 
                           index=test_data.loc[test_data.prop_review_score.isna()].index)

ls2_fill_series_test = pd.Series(ls2_fill_test, 
                            index=test_data.loc[test_data.prop_location_score2.isna()].index)

dd_fill_series_test = pd.Series(dd_fill_test, 
                           index=test_data.loc[test_data.orig_destination_distance.isna()].index)

fill_cols = ['comp2_rate', 'comp2_inv', 'comp3_rate', 'comp3_inv', 'comp5_rate', 'comp5_inv', 'comp8_rate', 'comp8_inv']
test_data.prop_review_score.fillna(rs_fill_series_test, inplace=True)
test_data.prop_location_score2.fillna(ls2_fill_series_test, inplace=True)
test_data.orig_destination_distance.fillna(dd_fill_series_test, inplace=True)
for c in fill_cols:
  test_data[c].fillna(0, inplace=True)

In [11]:
#pred_prob_y_booking = forest_booking.predict_proba(test_data)

#pred_booking_df = pd.DataFrame(map(lambda x: x[1], pred_prob_y_booking), columns=['booking_prob'])
#pred_booking_df['srch_id'] = test_data.srch_id
#pred_booking_df['prop_id'] = test_data.prop_id

#sorted_prop_id_booking = []
#for name, group in pred_booking_df.groupby('srch_id'):
  #for each search id, return the results of predictions, sorted by highest probability of booking
#  sorted_prop_id_booking.append(group.sort_values('booking_prob', ascending=False))

#uncomment this if you want to re-save the calculated bookings
#joblib.dump(sorted_prop_id_booking, '/content/datadrive/MyDrive/sortedp_prop_id_booking.joblib')

In [24]:
#sorted_prop_id_booking = joblib.load('/content/datadrive/MyDrive/sortedp_prop_id_booking.joblib')

In [13]:
pred_prob_y_click = forest_click.predict_proba(test_data)

pred_click_df = pd.DataFrame(map(lambda x: x[1], pred_prob_y_click), columns=['click_prob'])
pred_click_df['srch_id'] = test_data.srch_id
pred_click_df['prop_id'] = test_data.prop_id

sorted_prop_id_click = []
for name, group in pred_click_df.groupby('srch_id'):
  #for each search id, return the results of predictions, sorted by highest probability of click
  sorted_prop_id_click.append(group.sort_values('click_prob', ascending=False))

In [17]:
#joblib.dump(sorted_prop_id_click, '/content/datadrive/MyDrive/sorted_prop_id_click.joblib')

['/content/datadrive/MyDrive/sorted_prop_id_click.joblib']

In [62]:
results = []

#merge the two lists based on booking probability
for bookings, clicks in zip(sorted_prop_id_booking, sorted_prop_id_click):
  nonzero_book_prob = bookings[bookings.booking_prob > 0]
  nonzero_click_prob = []
  if nonzero_book_prob.shape[0] != bookings.shape[0]: #only find best click option if there are bookings with zero probability
    nonzero_click_prob = list(filter(lambda prop: prop.prop_id not in nonzero_book_prob.prop_id.values, clicks.to_records()))
  results.append(list(nonzero_book_prob.to_records()) + nonzero_click_prob)


In [64]:
results[:3]

[[(5, 0.4, 1, 28181),
  (4, 0.36, 1, 24194),
  (16, 0.35, 1, 74045),
  (18, 0.31, 1, 82231),
  (6, 0.3, 1, 34263),
  (13, 0.28, 1, 63894),
  (24, 0.26, 1, 123675),
  (21, 0.25, 1, 94729),
  (1, 0.21, 1, 5543),
  (22, 0.21, 1, 95031),
  (23, 0.19, 1, 99484),
  (20, 0.18, 1, 90385),
  (10, 0.18, 1, 56050),
  (9, 0.17, 1, 54937),
  (15, 0.16, 1, 73666),
  (25, 0.16, 1, 128085),
  (8, 0.13, 1, 50162),
  (17, 0.13, 1, 78599),
  (0, 0.12, 1, 3180),
  (14, 0.1, 1, 72090),
  (12, 0.1, 1, 61934),
  (3, 0.1, 1, 22393),
  (7, 0.09, 1, 37567),
  (27, 0.08, 1, 134992),
  (2, 0.07, 1, 14142),
  (28, 0.07, 1, 139162),
  (19, 0.05, 1, 89466),
  (11, 0.05, 1, 61632),
  (26, 0.04, 1, 128871)],
 [(30, 0.37, 3, 11425),
  (53, 0.32, 3, 131940),
  (34, 0.27, 3, 32382),
  (47, 0.24, 3, 91337),
  (40, 0.2, 3, 47377),
  (43, 0.19, 3, 71543),
  (29, 0.18, 3, 6533),
  (45, 0.17, 3, 80734),
  (38, 0.16, 3, 46861),
  (41, 0.14, 3, 63039),
  (39, 0.14, 3, 47238),
  (55, 0.13, 3, 132637),
  (49, 0.12, 3, 96614),
  (

In [70]:
with open('submission.csv', 'w') as submission_file:
  submission_writer = csv.writer(submission_file, delimiter=',')
  submission_writer.writerow(['srch_id', 'prop_id'])
  for search in results:
    for row in search:
      submission_writer.writerow([row[2], row[3]])

TypeError: ignored